In [20]:
# Dependencies
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from statsmodels.tsa.api import VAR
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Load data (FRED-MD)
df = pd.read_csv('../current.csv')

# Remove the first row (transformation codes)
transformation_codes = df.iloc[0]  # Transformation codes can be applied if needed
df = df.iloc[1:]

# Set the first column as the index and datetime
df.set_index(df.columns[0], inplace=True)
df.index = pd.to_datetime(df.index)

# Dropna
data = df.dropna()

# Create train data and target
target = (data['CPIAUCSL'].diff(12) / data['CPIAUCSL'].shift(12)) * 100
target = target.shift(-12).dropna()

# Difference all data (except CPIAUCSL)
data = data.diff().dropna()
data['CPIAUCSL'] = target
data = data.dropna()


In [22]:
# Forecast 
for forecast_horizon in range(4, 12 * 5 + 4, 12):
    # Get training data
    train_data = data.copy()[:-forecast_horizon]
    # Apply PCA
    variance = 0.95
    pca = PCA(n_components=variance)
    pca.fit(train_data)
    data_pca = pca.transform(train_data)
    train_data = pd.DataFrame(train_data)
    train_data['CPIAUCSL'] = target.iloc[:-forecast_horizon]
    for context_window in [1, 2, 4, 6, 12, 24]:
        # Split data into train and test sets
        cutoff = len(data) - forecast_horizon
        train_data = data.iloc[:cutoff]

        # Fit the VAR model
        model = VAR(train_data)
        results = model.fit(context_window)

        # Forecast all variables
        forecast_input = train_data.values[-context_window:]
        forecast_all = results.forecast(y=forecast_input, steps=forecast_horizon)

        # Extract CPIAUCSL forecasts and true values
        cpi_index = data.columns.get_loc("CPIAUCSL")
        y_pred = forecast_all[:, cpi_index]
        y_true = data.iloc[-forecast_horizon:]["CPIAUCSL"].values

        # Compute metrics
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        mae = mean_absolute_error(y_true, y_pred)

        # Log
        fp = "var.csv"
        var_log = pd.read_csv(fp) # forecast_horizon,context_window,dropout_rate,rmse,mae,forecast,true_vals
        new_row = {
            'forecast_horizon': forecast_horizon,
            'context_window': context_window,
            'variance': variance,
            'rmse': rmse,
            'mae': mae,
            'forecast': list(y_pred),
            'actual': list(y_true)
        }
        var_log = pd.concat([var_log, pd.DataFrame([new_row])], ignore_index=True)
        var_log.to_csv(fp, index=False)

/home/js/macroeconvue/modeling/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/tmp/ipykernel_737613/2838530968.py:46: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  var_log = pd.concat([var_log, pd.DataFrame([new_row])], ignore_index=True)
/home/js/macroeconvue/modeling/.venv/lib/python3.12/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/home/js/macroeconvue/modeling/.venv/